<a href="https://colab.research.google.com/github/nancy-kataria/NexTrade/blob/main/buyer_to_seller_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import kagglehub
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
path = kagglehub.dataset_download("walimuhammadahmad/receipt-data")

print("Path to dataset files:", path)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Path to dataset files: /root/.cache/kagglehub/datasets/walimuhammadahmad/receipt-data/versions/1


In [15]:
b2b_sales_data = pd.read_csv(path+"/data.csv")
b2b_sales_data.head()

<ipython-input-15-deec2f308238>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_sales_data = pd.read_csv(path+"/data.csv")


,NTN,NAME,S.T.Reg. No,Address,Customer_Receipt_NO,Customer_NTN,Customer_NAME,Customer_S.T.Reg. No,Customer_Phone Number,Customer_Address,...,Amount Including Taxes_Product_2,Amount Excluding Taxes_Product_3,Sales Tax @ 18%_Product_3,Amount Including Taxes_Product_3,Amount Excluding Taxes_Product_4,Sales Tax @ 18%_Product_4,Amount Including Taxes_Product_4,Total Amount Excluding Taxes,Total Sales Tax @ 18%,Total Amount Including Taxes
0,3650292853471,MUHAMMAD BEHZAD ARSHAD,10000-10046-8761-413-23,040-Tando Allah Main,1122916,0000092-2,ABDUL MANAN,32-77-8761-413-23,685874327,145-C Sattelite Town,...,5097.60,2046.0,368.28,2414.28,4929.0,887.22,5816.22,13195.0,2375.10,15570.10
1,3740503017362,MISS FARHAT MALIK,10000-10046-8761-413-23,118. Gulshan-E-Iqbal,1181647,0000094-9,MUNAWAR SULTANA,33-77-8761-413-23,744080328,Wala Colony House 34,...,1699.20,2272.0,408.96,2680.96,1870.0,336.60,2206.60,10826.0,1948.68,12774.68
2,3730122452353,MIRZA SHAHBAZ BAIG,10000-10046-8761-413-23,131-West Wood Colony,1196218,0000098-1,EHSAN SHAFQAT,34-77-8761-413-23,912263285,Apt-906 Rehman Tower,...,817.74,3404.0,612.72,4016.72,2304.0,414.72,2718.72,10958.0,1972.44,12930.44
3,3650292844921,MUHAMMAD BILAL ARSHAD,10000-10046-8761-413-23,145-C Sattelite Town,908244,0000131-7,MUHAMMAD ANEES QURESHI,35-77-8761-413-23,3003232696,37-C P. O Box. 20021,...,1438.42,3186.0,573.48,3759.48,2904.0,522.72,3426.72,7377.0,1327.86,8704.86
4,3650170698023,BABAR HABIB NOON,10000-77-18730-413-23,14-Km Ferozpure Road,902983,0000146-5,MUHAMMAD AJMAL,36-77-8761-413-23,3005352693,205 Muhammadia Plaza,...,4874.58,4559.0,820.62,5379.62,5734.0,1032.12,6766.12,20586.0,3705.48,24291.48


In [16]:
b2b_sales_data.dtypes

,0
NTN,object
NAME,object
S.T.Reg. No,object
Address,object
Customer_Receipt_NO,int64
Customer_NTN,object
Customer_NAME,object
Customer_S.T.Reg. No,object
Customer_Phone Number,object
Customer_Address,object


In [17]:
# checking the mixed datatypes in a colummn
b2b_sales_data['NTN'].apply(type).value_counts()

,count
NTN,
<class 'str'>,16384
<class 'int'>,3616


In [18]:
# changing the int data types to string
b2b_sales_data['NTN'] = b2b_sales_data['NTN'].astype(str)
# checking the mixed datatypes in a colummn
b2b_sales_data['NTN'].apply(type).value_counts()

,count
NTN,
<class 'str'>,20000


In [19]:
b2b_sales_data['Customer_NTN'].apply(type).value_counts()

,count
Customer_NTN,
<class 'str'>,20000


In [20]:
b2b_sales_data = b2b_sales_data.dropna(subset=['NAME'])

In [21]:
b2b_sales_data = b2b_sales_data.dropna(subset=['Customer_NAME'])

In [22]:
b2b_sales_data['Product 1'] = b2b_sales_data['Product 1'].fillna('Unknown Product')
b2b_sales_data['Product 2'] = b2b_sales_data['Product 2'].fillna('Unknown Product')
b2b_sales_data['Product 3'] = b2b_sales_data['Product 3'].fillna('Unknown Product')
b2b_sales_data['Product 4'] = b2b_sales_data['Product 4'].fillna('Unknown Product')

In [23]:
b2b_sales_data.drop_duplicates(inplace=True)
b2b_sales_data = b2b_sales_data.drop(['Address','Customer_Address', 'Customer_Phone Number'], axis=1)

In [24]:
normalized_data = []
for i in range(1, 5):
    temp = pd.DataFrame({
        'NTN': b2b_sales_data['NTN'],
        'NAME': b2b_sales_data['NAME'],
        'S.T.Reg. No': b2b_sales_data['S.T.Reg. No'],
        'Customer_Receipt_NO': b2b_sales_data['Customer_Receipt_NO'],
        'Customer_NTN': b2b_sales_data['Customer_NTN'],
        'Customer_NAME': b2b_sales_data['Customer_NAME'],
        'Customer_S.T.Reg. No': b2b_sales_data['Customer_S.T.Reg. No'],
        'Business Name': b2b_sales_data['Business Name'],
        'Date': b2b_sales_data['Date'],
        'Product': b2b_sales_data[f'Product {i}'],
        'Quantity': b2b_sales_data[f'Quantity_Product_{i}'],
        'Rate': b2b_sales_data[f'Rate_Product_{i}'],
        'Tax': b2b_sales_data[f'Tax_Product_{i}'],
        'Price_with_Tax': b2b_sales_data[f'Product_{i}_Price_with_Tax'],
        'Amount Excluding Taxes': b2b_sales_data[f'Amount Excluding Taxes_Product_{i}'],
        'Sales Tax @ 18%': b2b_sales_data[f'Sales Tax @ 18%_Product_{i}'],
        'Amount Including Taxes': b2b_sales_data.get(f'Amount Including Taxes_Product_{i}', None)
    })

    normalized_data.append(temp)

# Combine all 4 blocks vertically
normalized_df = pd.concat(normalized_data, ignore_index=True)

# Reset index
normalized_df.reset_index(drop=True, inplace=True)

In [25]:
# Loading of English NLP pipeline for lemmatization
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

def clean_product_name(name):
    if pd.isnull(name):
        return ""

    # Turning text into lowercase
    name = name.lower()

    # Removing special characters, numbers, extra commas/spaces
    name = re.sub(r'[^a-zA-Z\s]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()

    # Tokenization and removal of stopwords
    tokens = [word for word in name.split() if word not in stop_words]

    # Lemmatization
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]

    # Return cleaned name
    return ' '.join(lemmas)

# applying the above function
normalized_df['Product'] = normalized_df['Product'].apply(clean_product_name)


In [ ]:
# The above step took 7 mins to clean